# NY Times API 

So far, we have had exposure to working with requests library in Python. Using our knwoledge of APIs, we are going to use the NY Times API and collect a set of data.Please note that the NY Times API will not return full text for available news article. However, it will provide us with useful metadata such as subject terms, abstract, and updated-date, urls to images, and any other information a developer might seek when scraping the site. 

The main steps for this tutotal are the following:
1. Fetch data from NY Times API
2. Load JSON data using requests
3. parse nested JSON for title of article and data updated
4. **Advanced**: Use a visualization library of your choice, visually represent the data. 

First step is to [sign up](https://developer.nytimes.com/signup) for getting an API key. Take a moment to look through the link. Notice that the available api-key requests are separated by the _type_ of request. This is helpful for us as developers but it also provides user meta data to NY Times. For example, client metadata from api-key requests can be used by NY Times for analysis ([BIA](https://en.wikipedia.org/wiki/Business_intelligence)), building future APIs, improving existing ones, or providing additional API wrappers. In an organizational setting, generated meta data from the client side is just as important as receiving data from the server. User data has helped NY Times improve its developer tools such as providing new APIs, corresponding wrappers, and improving data accessibility. 

In the sign-up page, fill out your information to receive your api-key. For the purposes of this tutorial, the Top Stories API key was requested and used. Feel free to pick any of the other available APIs. See a [list](https://developer.nytimes.com/) of available APIs to follow through this tutorial. Once you have received the api key in your e-mail, store it as a variable in your python script. 

One of the main reasons the NY Times was selected for the purposes of this course is to receive a visual understanding between dynamic vs. static URL paramters. , URL To understand how a request can be made (GET request) NY Times has made a [GUI interaface](https://developer.nytimes.com/top_stories_v2.json) available. 



In [29]:
import requests
#sign up and get a key -- I chose Top Stories
key = "a1b6d14c03494629990937518357c475"
# within Top Stories, there are list of parameters to work with. Choose one param or a list. 
url = 'https://api.nytimes.com/svc/topstories/v2/politics.json?q=&api-key=%s' % key
print(url)
r = requests.get(url)



https://api.nytimes.com/svc/topstories/v2/politics.json?q=&api-key=a1b6d14c03494629990937518357c475
